# Desarrollo de producto de datos

# Servidor

Alumno: Alejandro Méndez Miranda

Para este trabajo se tomó como referencia el notebook analizado en clases. Se tomó gran parte del código pero con las modificaciones para responder a las preguntas.

## Carga de librerías

Para ejecutar el trabajo de Servidor se cargan las librerías para análisis de imágenes y para la generación del servidor.

In [1]:
import os
import io
import cv2
import requests
import numpy as np
from IPython.display import Image, display

import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox


import io
import uvicorn
import numpy as np
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse

## Creación del servidor

Se reutilizó el código visto en clases pero con pequeñas modificaciones.

Primero se cortó la función de prediction para poder reutilizar el código. Este código nos permitía tanto generar la predicción de box en las imágenes y seleccionar su categoría como también la posibilidad de contabilizar cada una de las categorías presentes en la imagen. A esta función se le agregó el parámetro de confianza que será útil en las funciones siguientes; y un parámetro para hacer la consulta de cantidad de box por categoría indicada o si se deja como un string vacío entrega todas las categorías.

In [2]:
app = FastAPI(title='Implementando un modelo de Machine Learning usando FastAPI')

class Model(str, Enum):
    yolov3tiny = "yolov3-tiny"
    yolov3 = "yolov3"
    
#Se separó la función predictión para no ser llamada durange el llamado de un cliente.
def prediction(model: Model, confidence = 0.5,countobject = None, file: UploadFile = File(...)):

    """La función contiene los parámetros:
    model: Modelo a utilizar, se selecciona a partir de la clase creada Model
    confidence: Nivel de convianza al momento de generar las detecciones
    countobject: None como defecto para no afectar en el caso de que no se
                 requiera el conteo de categorías. Util para cuando se quiera
                 hacer el llamado al endpoint countObjects
    file: Archivo a predecir
    
    La función retorna:
    Si endpoint es predict --> Imagen con las predicciones realizadas
    Si endpoint es countObjects/countSheeps --> Conteo de categorías en formato string"""
    
    
    filename = file.filename
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Tipo de archivo no soportado.")
    
    #La función se asegura de que confidence sea un valor numérico entre 0 y 1
    #, en caso de no serlo retorna excepciones con los errores correspondientes.
    
    try:
        confidence = float(confidence)
        if confidence >= 1 or confidence <= 0:
             raise HTTPException(status_code=415, detail="El valor tiene que estar entre 0 y 1")
    except:
        raise HTTPException(status_code=415, detail="El valor tiene que ser numérico entre 0 y 1")

    

    image_stream = io.BytesIO(file.file.read())
    image_stream.seek(0)
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    
    

    bbox, label, conf = cv.detect_common_objects(image, confidence = confidence, model=model)
    label2 = np.array(label)
    output_image = draw_bbox(image, bbox, label, conf)
    cv2.imwrite(f'images_uploaded/{filename}', output_image)
    
    
    file_image = open(f'images_uploaded/{filename}', mode="rb")

    #Creacion de condicionales para generar los dos posibles casos de endpoints
    #Si el endpoint es predict countobject = None, por lo que que entrará en el
    #condicional else y retornará la imagen con las predicciones.
    #En el caso que sea un string vacío, como el caso del endpoint countObject
    #por defecto, entregará la lista completa con los conteos; en caso de que 
    #se señale un objeto en particular, realizará el conteo de este.
    
    if countobject == "":
        count_image =""
        if len(np.unique(label2)) >0:
            for i in np.unique(label2):
                count_image += f"||  Object: {i:8s}  | " + f"Count: {label.count(i)} ||"
            return(count_image)
        else:
            return("  No object detected")
    elif countobject is not None:        
        count_image = f"  |  Object: {countobject}  | " + f"Count: {label.count(countobject)} |"
        return(count_image)
    else:
        # Retornar la imagen como un stream usando un formato específico
        return(file_image)
    
    
    
@app.get("/")
def home():
    return "¡Felicitaciones! Tu API está funcionando según lo esperado. Anda ahora a http://localhost:8000/docs."

#El endpoint predict entrega la misma funcionalidad que la vista en clases pero existe el nuevo parámetro
#confidence para ser entregado al modelo
@app.post("/predict") 
def results2(model: Model, confidence = 0.5, file: UploadFile = File(...)):
    file_image=prediction(model=model, confidence = confidence, file = file)

    return StreamingResponse(file_image, media_type="image/jpeg")


#Nueva función para realizar el conteo. Contiene nuevos parámetros como confidence y countobject como string
#vacío por defecto. Retorna el conteo de la/las categoría(s)
@app.post("/countObjects") 
def countobjects(model: Model, confidence = 0.5, countobject = "", file: UploadFile = File(...)):
    if type("countobject") != str:
        raise HTTPException(status_code=415, detail="El valor tiene que ser formato string")
    else:
        count_image =prediction(model = model, confidence = confidence, countobject = countobject, file = file)
    return(count_image)

#Finalmente se puede utilizar la función para crear un endpoint específico. En este caso para
#contar ovejas 
@app.post("/countSheeps") 
def countobjects(model: Model, confidence = 0.5, file: UploadFile = File(...)):
    if type("countobject") != str:
        raise HTTPException(status_code=415, detail="El valor tiene que ser formato string")
    else:
        count_image =prediction(model = model, confidence = confidence, countobject = "sheep", file = file)
    return(count_image)

In [ ]:
nest_asyncio.apply()
host = "127.0.0.1"
uvicorn.run(app, host=host, port=8000)